### AO3 History Downloader

[what does this do]

### How to Use

#### Provide Your AO3 Login Info

In order to retrieve your stats from AO3, you will need to provide your username and your _otwarchive_session cookie to fill in the USERNAME and COOKIE fields on lines 2 and 3 of the cell below (you will need to keep the quotes around your username and cookie).

In Chrome, you can find your cookie value as follows:
1. Load any archiveofyourown.org page.
2. Right click anywhere on the page and select "Inspect" from the menu.
3. On the upper menu bar, click Application.
4. Find the row in the table with Name _otwarchive_session. Copy the contents of Value into the below cell in place of COOKIE (keep the quotes).

If you see the "Login error" message, then the app wasn't able to log in using the username/cookie you provided. The most likely reason for this is that the cookie has refreshed and has a different value now. You should repeat the above steps and copy the new value into the COOKIE field. 

#### Specify start and end pages

To avoid overloading AO3 with too many requests, please specify which pages in your history you want to save. So if you're only interested in your reading history from 2020, find the first and last pages (e.g. page 2 and page 10) in your history that contain fics you read in 2020, and fill them in on lines 4 and 5 below (start_page and end_page).

If you only want to download the data from a single page in your history, set both start_page and end_page to that page number.

#### Run the Script

Once you've filled in your login info, go to "Cell" -> "Run All" in the menu. That should start downloading your AO3 history. Watch out for any login errors. If you have other issues you can contact winglavender@gmail.com and I'll try to address them. 

#### Runtime 

The whole process will take longer the more pages of history you want to download. It's relatively quick but not instantaneous. In my experience, it takes a couple of minutes to go through ~50 pages of history.

#### Mobile

This has not been tested on mobile. I'd recommend running the downloader from a computer.

#### Timeout Warning

You may see the following warning while the script is running: "timeout... waiting 3 mins and trying again." This happens sometimes when you make too many server requests to AO3 in a short period of time. In most cases the script should continue running after 3 minutes. If this continues to happen repeatedly, try taking a break and coming back in an hour or so to try again.

### Warning: Download Your Data!

If you're running this notebook in Binder on your browser, be aware that your session will time out after only 10 minutes of inactivity. So once the script has finished running, you should download the results file right away - if you let it sit for too long, the notebook will shut down and you'll have to run it again from script in order to get your results.

In [ ]:
### MODIFY THIS SECTION ONLY ###
username = 'USERNAME'
cookie = 'COOKIE'
start_page = 1
end_page = 3
################################# 

from ao3 import AO3
import requests
req = requests.Session()
api = AO3()
api.login(username, cookie)

# check if login was successful
login_success = False
try:
    rh = api.user.reading_history()
    item = next(rh)
    print("Login successful!")
    login_success = True
except AttributeError:
    print("Login error. Please check your username and cookie and try again.")

import csv
import time
import datetime
from IPython.display import FileLink
filename = ""
if login_success:
    header = ['work_id', 'date', 'numvisits', 'title', 'author', 'fandom', 'warnings', 'relationships', 'characters', 'additional_tags', 'words', 'chapters', 'comments', 'kudos', 'bookmarks', 'hits', 'pubdate']
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H:%M:%S')
    filename = 'history_' + st + '.csv'

    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter='\t', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(header)
        for work in api.user.reading_history(start_page, end_page):
            row = []
            for elem in work:
                if type(elem) is list:
                    row.append(",".join(elem))
                else:
                    row.append(elem)
            writer.writerow(row)
    print("\n\nDone!\nRight-click the link below and select 'Save link as' to save the results file to your computer.")
FileLink(filename)